In [3]:
%defaultDatasource jdbc:h2:mem:db

In [4]:
DROP TABLE IF EXISTS Ansiedade;
DROP TABLE IF EXISTS Depressao;
DROP TABLE IF EXISTS MentalFacilities;

CREATE TABLE Depressao(
    nomePais VARCHAR(52),
    anoPesquisa varchar(16),
    populacao bigint,
    porcentagemHomens FLOAT,
    porcentagemMulheres FLOAT,
) AS SELECT
    Entity,
    Year, 
    Population,
    Prevalence_Depressive_Disorders_Male,
    Prevalence_Depressive_Disorders_Female
FROM CSVREAD('../data/depression_mh.csv');

CREATE TABLE MentalFacilities(
    nomePais VARCHAR(52),
    anoPesquisa VARCHAR(16),
    hospitais FLOAT,
    servicos_de_saude FLOAT,
    servicos_de_saude_nao_gov FLOAT,
    servicos_de_saude_day_threat FLOAT,
    comunidade_residencial FLOAT
) AS SELECT
    Country,
    Year, 
    Mental_hospitals,
    Mental_health_units,
    Mental_health_outpatient,
    Mental_health_day_treatment,
    Community_residential_facilities
FROM CSVREAD('../data/mental_health_facilities_gho.csv');

In [5]:
DELETE FROM Depressao D WHERE D.porcentagemHomens IS NULL;
--SELECT * FROM Depressao;

In [6]:
UPDATE MentalFacilities SET hospitais = 0 WHERE hospitais IS NULL;
UPDATE MentalFacilities SET servicos_de_saude = 0 WHERE servicos_de_saude IS NULL;
UPDATE MentalFacilities SET servicos_de_saude_nao_gov = 0 WHERE servicos_de_saude_nao_gov IS NULL;
UPDATE MentalFacilities SET servicos_de_saude_day_threat = 0 WHERE servicos_de_saude_day_threat IS NULL;
UPDATE MentalFacilities SET comunidade_residencial = 0 WHERE comunidade_residencial IS NULL;

SELECT * FROM MentalFacilities;

DROP TABLE IF EXISTS MentalFacilities_v2;
CREATE TABLE MentalFacilities_v2 (
    nomePais VARCHAR(52),
    anoPesquisa INT,
    porcentagem FLOAT
)

In [7]:
INSERT INTO MentalFacilities_v2 (nomePais, anoPesquisa, porcentagem)
SELECT MF.nomePais, MF.anoPesquisa, (MF.hospitais + MF.servicos_de_saude + MF.servicos_de_saude_nao_gov + MF.servicos_de_saude_day_threat + MF.comunidade_residencial)
FROM MentalFacilities MF WHERE CAST(SUBSTRING(MF.anoPesquisa, 1, 4) AS INT) = MF.anoPesquisa;

In [11]:
SELECT * FROM MentalFacilities_v2;

In [46]:
DROP TABLE IF EXISTS Juncao; 
CREATE TABLE Juncao (
    nomePais VARCHAR(53),
    anoPesquisa int,
    porcentagemHomens FLOAT,
    porcentagemMulheres FLOAT,
    porcentagemInstalacoes FLOAT
);

In [47]:
DROP VIEW IF EXISTS MAX_ANO;
CREATE VIEW MAX_ANO AS
SELECT nomePais, MAX(anoPesquisa) as ano_max FROM MentalFacilities_v2
GROUP BY nomePais, anoPesquisa;

--SELECT * FROM MAX_ANO;

In [48]:
INSERT INTO Juncao (nomePais, anoPesquisa, porcentagemHomens, porcentagemMulheres, porcentagemInstalacoes)
SELECT MF.nomePais, D.anoPesquisa, ROUND(D.porcentagemHomens , 4), ROUND(D.porcentagemMulheres, 4), ROUND(MF.porcentagem, 3)
FROM MentalFacilities_v2 MF, Depressao D, MAX_ANO MA
WHERE MF.nomePais = D.nomePais AND MF.nomePais = MA.nomePais AND MF.anoPesquisa = MA.ano_max AND D.anoPesquisa = MA.ano_max;
SELECT * FROM Juncao;

In [65]:
SELECT ROUND(SUM(porcentagemHomens)/124 , 4) as homens, ROUND(SUM(porcentagemMulheres)/124 ,4) as mulheres FROM Juncao;

In [56]:
SELECT * FROM Juncao ORDER BY porcentagemInstalacoes DESC;

In [63]:
DROP TABLE IF EXISTS Caso01;
CREATE TABLE Caso01(
    nomeRange VARCHAR(52),
    porcentagemDMedia DOUBLE,
    porcentagemIMedia DOUBLE
);

INSERT INTO Caso01 (porcentagemDMedia, porcentagemIMedia)
SELECT ROUND(AVG(J.porcentagemHomens + J.porcentagemMulheres), 4), ROUND(AVG(J.porcentagemInstalacoes), 4)
FROM Juncao J WHERE J.porcentagemInstalacoes > 8;
UPDATE Caso01 SET nomeRange = 'High Facilities Percentage' WHERE porcentagemIMedia > 8;

INSERT INTO Caso01 (porcentagemDMedia, porcentagemIMedia)
SELECT ROUND(AVG(J.porcentagemHomens + J.porcentagemMulheres), 4), ROUND(AVG(J.porcentagemInstalacoes), 4)
FROM Juncao J WHERE J.porcentagemInstalacoes < 8 AND J.porcentagemInstalacoes > 1.9;
UPDATE Caso01 SET nomeRange = 'Medium Facilities Percentage' WHERE porcentagemIMedia < 8 AND porcentagemIMedia > 1.9;

INSERT INTO Caso01 (porcentagemDMedia, porcentagemIMedia)
SELECT ROUND(AVG(J.porcentagemHomens + J.porcentagemMulheres), 4), ROUND(AVG(J.porcentagemInstalacoes), 4)
FROM Juncao J WHERE J.porcentagemInstalacoes < 1.9;
UPDATE Caso01 SET nomeRange = 'Low Facilities Percentage' WHERE porcentagemIMedia < 1.9;

In [64]:
SELECT * FROM CASO01;